In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from google.colab import drive
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, GlobalAveragePooling3D, Dropout

# Define paths
strike_images_path = "/content/drive/My Drive/HG-R1-1-S/images/"
strike_masks_path = "/content/drive/My Drive/HG-R1-1-S/masks/"
neutral_images_path = "/content/drive/My Drive/HG-R1-1-N/images/"
neutral_masks_path = "/content/drive/My Drive/HG-R1-1-N/masks/"

drive.mount('/content/drive')

# Use mixed precision to reduce memory usage
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load frames and masks from a folder
def load_frames_and_masks(images_path, masks_path, label):
    frame_paths = sorted([os.path.join(images_path, f) for f in os.listdir(images_path) if f.endswith('.jpg') or f.endswith('.png')])
    mask_paths = sorted([os.path.join(masks_path, f) for f in os.listdir(masks_path) if f.endswith('.png')])

    frames = [cv2.imread(frame_path) for frame_path in frame_paths]
    masks = [cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE) for mask_path in mask_paths]

    frames_with_masks = []
    for frame, mask in zip(frames, masks):
        mask_expanded = np.expand_dims(mask, axis=-1)
        combined = np.concatenate((frame, mask_expanded), axis=-1)
        frames_with_masks.append(combined)

    labels = [label] * len(frames_with_masks)
    return np.array(frames_with_masks), np.array(labels)

# Load data from both strike and neutral folders
strike_data, strike_labels = load_frames_and_masks(strike_images_path, strike_masks_path, 1)  # 1 for strike
neutral_data, neutral_labels = load_frames_and_masks(neutral_images_path, neutral_masks_path, 0)  # 0 for neutral


In [ ]:
# Combine without extra shuffling
X = np.concatenate((strike_data, neutral_data), axis=0)
y = np.concatenate((strike_labels, neutral_labels), axis=0)

# Normalize pixel values
X = X.astype('float32') / 255.0
y = to_categorical(y)  # One-hot encode labels for binary classification

# Define sequence length
sequence_length = 5

# Group frames into sequences
num_sequences = X.shape[0] // sequence_length
X = X[:num_sequences * sequence_length].reshape(
    (num_sequences, sequence_length, X.shape[1], X.shape[2], X.shape[3])
)
y = y[:num_sequences * sequence_length:sequence_length]  # Use one label per sequence

# Define generator function for on-the-fly augmentation
def data_generator(X, y, batch_size, sequence_length):
    while True:
        for i in range(0, len(X), batch_size):
            batch_X = X[i:i + batch_size]
            batch_y = y[i:i + batch_size]
            yield np.array(batch_X), np.array(batch_y)

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.21052631578, random_state=42)

In [ ]:
# Build the model
model = Sequential()
model.add(Conv3D(16, (2, 2, 2), activation='relu', input_shape=(sequence_length, X.shape[2], X.shape[3], X.shape[4])))
model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), padding='same'))

# Simplify the architecture with fewer layers
model.add(GlobalAveragePooling3D())

# Fully connected layers with regularization and dropout
model.add(Dense(32, activation='relu', kernel_regularizer='l2'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))  # Binary classification with softmax

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train using generator
batch_size = 8
steps_per_epoch = len(X_train) // batch_size

history = model.fit(
    data_generator(X_train, y_train, batch_size, sequence_length),
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=(X_val, y_val)
)

# Evaluate the model
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=1)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")


Epoch 1/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 19s 4s/step - accuracy: 0.4427 - loss: 0.9223 - val_accuracy: 0.3750 - val_loss: 0.9353
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 651ms/step - accuracy: 0.5584 - loss: 0.9137 - val_accuracy: 0.3750 - val_loss: 0.9281
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 890ms/step - accuracy: 0.4018 - loss: 0.9294 - val_accuracy: 0.3750 - val_loss: 0.9210
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 694ms/step - accuracy: 0.5312 - loss: 0.9132 - val_accuracy: 0.3750 - val_loss: 0.9121
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 970ms/step - accuracy: 0.5938 - loss: 0.8893 - val_accuracy: 0.3750 - val_loss: 0.9041
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 765ms/step - accuracy: 0.5130 - loss: 0.8966 - val_accuracy: 0.3750 - val_loss: 0.8969
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 806ms/step - accuracy: 0.3791 - loss: 0.8956 - val_accuracy: 0.3750 - val_loss: 0.8903
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 706ms/step - accuracy: 0.7003 - loss: 0.8749 - val_accuracy: 0.3750 - val_loss: 0